# Кибериммунная автономность$\\$Создание конструктивно защищённого автономного наземного транспортного средства$\\$Модуль 2

## О документе

Версия 1.03

Модуль 2 для регионального этапа соревнований по кибериммунной автономности

## Подготовка к работе c модулем 2

Перенесите в блок ниже ваши реализации классов коммуникационного шлюза (CommunicationGateway), систему навигации (NavigationSystem), управления (ControlSystem)

In [17]:
from src.control_system import BaseControlSystem
from src.config import SERVOS_QUEUE_NAME
from src.navigation_system import BaseNavigationSystem
from multiprocessing import Queue
from src.communication_gateway import BaseCommunicationGateway
from src.config import CONTROL_SYSTEM_QUEUE_NAME
from src.event_types import Event
class CommunicationGateway(BaseCommunicationGateway):
    """CommunicationGateway класс для реализации логики взаимодействия
    с системой планирования заданий

    Работает в отдельном процессе, поэтому создаётся как наследник класса Process
    """
    def _send_mission_to_consumers(self):
        """ метод для отправки сообщения с маршрутным заданием в систему управления """

        # имена очередей блоков находятся в файле src/config.py
        # события нужно отправлять в соответствие с диаграммой информационных потоков
        control_q_name = CONTROL_SYSTEM_QUEUE_NAME

        # события передаются в виде экземпляров класса Event,
        # описание класса находится в файле src/event_types.py
        event = Event(source=BaseCommunicationGateway.event_source_name,
                      destination=control_q_name,
                      operation="set_mission", parameters=self._mission
                    )

        # поиск в каталоге нужной очереди (в данном случае - системы управления)
        control_q: Queue = self._queues_dir.get_queue(control_q_name)
        # отправка события в найденную очередь
        control_q.put(event)


class ControlSystem(BaseControlSystem):
    """ControlSystem блок расчёта управления """

    def _send_speed_and_direction_to_consumers(self, speed, direction):
        servos_q_name = SERVOS_QUEUE_NAME  # замените на правильное название очереди
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # отправка сообщения с желаемой скоростью
        event_speed = Event(source=BaseControlSystem.event_source_name,
                            destination=servos_q_name,
                            operation="set_speed", parameters=speed
                            )

        # отправка сообщения с желаемым направлением
        event_direction = Event(source=BaseControlSystem.event_source_name,
                                destination=servos_q_name,
                                operation="set_direction", parameters=direction
                                )

        servos_q.put(event_speed)
        servos_q.put(event_direction)

    # найдите подходящее имя очереди в файле src/config.py и добавьте в строчку ниже

class NavigationSystem(BaseNavigationSystem):
    """ класс навигационного блока """
    def _send_position_to_consumers(self):
        control_q_name = CONTROL_SYSTEM_QUEUE_NAME # замените на правильное название очереди
        event = Event(source=NavigationSystem.event_source_name,
                      destination=control_q_name,
                      operation="position_update", parameters=self._position
                    )
        control_q: Queue = self._queues_dir.get_queue(control_q_name)
        control_q.put(event)

Если у вас настроена и работает СУПА, установите в True значение переменной afcs_present

In [18]:
afcs_present = True

В этом модуле изменим идентификатор машинки:

In [19]:
car_id = "m4"

### Модуль 2. Отправка маршрутного здания и следование по маршруту

В рамках этого модуля отрабатывается только автономность. 
Никаких киберпрепятствий пока не будет. 

Что нужно сделать по пунктам:
1. Для заданных начальных и конечных точек создать маршрут, содержащий не менее пяти промежуточных путевых точек вручную или с использованием таких инструментов как APMPlanner, MissionPlanner. <br>Содержимое wpl файла нужно вставить в соответствующий кодовый блок блокнота, запустить симуляцию и убедиться, что машинка успешно проходит этот маршрут.
2. Задать скоростные ограничения в виде массива элементов GeoSpecificSpeedLimit
3. Запустить симуляцию


Формат WPL файла описан на этой странице: 

https://mavlink.io/en/file_formats/#mission_plain_text_file

Пример содержимого файла с текстовым описанием маршрута

In [20]:
wpl_file_content =  """QGC WPL 110
0	1	0	16	0	5	0	0	59.8746946570238379	29.8298710584640503	0	1
1	0	3	16	0	5	0	0	59.8743984958028932	29.8298978805541992	20	1
2	0	3	16	0	5	0	0	59.8741157939945907	29.8296511173248291	20	1
3	0	3	16	0	5	0	0	59.8739380944847426	29.8291575908660889	20	1
4	0	3	16	0	5	0	0	59.8739004005272264	29.8288142681121826	20	1
5	0	3	16	0	5	0	0	59.8737011603275562	29.8287981748580933	20	1
6	0	3	16	0	5	0	0	59.8736876981088386	29.8291361331939697	20	1
7	0	3	16	0	5	0	0	59.8732999838707869	29.8294526338577271	20	1
8	0	3	16	0	5	0	0	59.8732192095021247	29.8293507099151611	20	1
9	0	3	16	0	5	0	0	59.873138434937232	29.8293828964233398	20	1
10	0	3	16	0	5	0	0	59.8721125808917094	29.829753041267395	20	1
11	0	3	16	0	5	0	0	59.8707985798712627	29.8302143812179565	20	1
12	0	3	16	0	5	0	0	59.8707931945143343	29.8310619592666626	20	1
13	0	3	16	0	5	0	0	59.8705508525499752	29.8311156034469604	20	1
14	0	3	16	0	5	0	0	59.8704727642074488	29.8297101259231567	20	1
15	0	3	16	0	5	0	0	59.8703650558478842	29.8288679122924805	20	1
16	0	3	16	0	5	0	0	59.8702061853806953	29.8282885551452637	20	1
17	0	3	16	0	5	0	0	59.8692421754226274	29.8292005062103271	20	1
"""


Для выполнения задания в блоке ниже  
1. замените имя файла с маршрутом (на свой), вот эту строку:
   ```python
   wpl_file = "module2.wpl"
   ```
2. измените список скоростных ограничений, вот этот массив:
   ```python
   speed_limits = [
    GeoSpecificSpeedLimit(...
    ```
   
Учтите, что вам нужно доставить груз как можно быстрее, но при этом не нарушить правила дорожного движения. 

Ограничения скорости
- населённый пункт - 60 км/ч
- пешеходная зона - 20 км/ч
- магистраль - 110 км/ч

Составьте маршрут доставки так, чтобы машинка доехала за минимальное время. Для определения оптимального маршрута допустимо пользоваться сторонними сервисами (например, Яндекс картами).

In [21]:
# создадим wpl файл и запишем туда маршрутное задание
wpl_file = "module2.wpl"

**Примечание**:
После создания собственного маршрута удалите перезапись содержимого, чтобы не потерять свои наработки!


In [22]:
with open(wpl_file, "w") as f:
    f.write(wpl_file_content)

**Напоминание**: не забудьте изменить скоростные ограничения (speed_limits) при задании маршрутного задания!

In [30]:
from src.wpl_parser import WPLParser

parser = WPLParser(wpl_file)    
points = parser.parse()
print(points)

from src.mission_type import GeoSpecificSpeedLimit
speed_limits = [
    GeoSpecificSpeedLimit(0, 20),
    GeoSpecificSpeedLimit(9, 60),
    GeoSpecificSpeedLimit(11, 20),
    GeoSpecificSpeedLimit(13, 60),
]

from src.mission_planner import Mission

home = points[0]
mission = Mission(home=home, waypoints=points,speed_limits=speed_limits, armed=True)

[Point(59.87469465702384, 29.82987105846405, 0.0), Point(59.87439849580289, 29.8298978805542, 0.0), Point(59.87411579399459, 29.82965111732483, 0.0), Point(59.87393809448474, 29.82915759086609, 0.0), Point(59.873900400527226, 29.828814268112183, 0.0), Point(59.873701160327556, 29.828798174858093, 0.0), Point(59.87368769810884, 29.82913613319397, 0.0), Point(59.87329998387079, 29.829452633857727, 0.0), Point(59.873219209502125, 29.82935070991516, 0.0), Point(59.87313843493723, 29.82938289642334, 0.0), Point(59.87211258089171, 29.829753041267395, 0.0), Point(59.87079857987126, 29.830214381217957, 0.0), Point(59.870793194514334, 29.831061959266663, 0.0), Point(59.870550852549975, 29.83111560344696, 0.0), Point(59.87047276420745, 29.829710125923157, 0.0), Point(59.870365055847884, 29.82886791229248, 0.0), Point(59.870206185380695, 29.828288555145264, 0.0), Point(59.86924217542263, 29.829200506210327, 0.0)]


Запускаем симуляцию с новым маршрутным заданием

In [31]:
from time import sleep
from geopy import Point as GeoPoint


from src.queues_dir import QueuesDirectory
from src.servos import Servos
from src.sitl import SITL
from src.cargo_bay import CargoBay
from src.mission_planner import MissionPlanner
from src.config import LOG_ERROR, LOG_INFO
from src.mission_planner_mqtt import MissionSender
from src.sitl_mqtt import TelemetrySender
from src.system_wrapper import SystemComponentsContainer


# координата текущего положения машинки
home = GeoPoint(59.8746946570238379,	29.8298710584640503)


# каталог очередей для передачи сообщений между блоками
queues_dir = QueuesDirectory() 

if afcs_present:
    mission_sender = MissionSender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)
    telemetry_sender = TelemetrySender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)

mission_planner = MissionPlanner(
    queues_dir, afcs_present=afcs_present, mission=mission)

sitl = SITL(
    queues_dir=queues_dir, position=home,
    car_id=car_id, post_telemetry=afcs_present, log_level=LOG_ERROR)


communication_gateway = CommunicationGateway(
    queues_dir=queues_dir, log_level=LOG_ERROR)
control_system = ControlSystem(queues_dir=queues_dir, log_level=LOG_INFO)

navigation_system = NavigationSystem(
    queues_dir=queues_dir, log_level=LOG_ERROR)

servos = Servos(queues_dir=queues_dir, log_level=LOG_ERROR)
cargo_bay = CargoBay(queues_dir=queues_dir, log_level=LOG_INFO)

# у нас получилось довольно много блоков, используем класс SystemComponentsContainer
# для упрощения рутинной работы с ними - таким образом мы собираем все блоки машинки в одном "кузове"
system_components = SystemComponentsContainer(
    components=[
        mission_sender,
        telemetry_sender,
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system
    ] if afcs_present else [
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system
    ])

system_components.start()

# ограничение поездки по времени
# параметр sleep - время в секундах,
# настройте этот параметр так, чтобы ваша машинка завершила маршрут
# в случае превышения времени выполнения ячейки на более чем 10 секунд от заданного, 
# допустимо перезапустить вычислительное ядро и повторно выполнить весь блокнот, штрафные очки за это не начисляются
# при условии, что повторный запуск закончился успешно
sleep(100)

# останавливаем все компоненты
system_components.stop()

# удалим все созданные компоненты
system_components.clean()

[ИНФО][QUEUES] создан каталог очередей
[ИНФО][QUEUES] регистрируем очередь planner.mqtt
[ИНФО][QUEUES] регистрируем очередь sitl.mqtt
[ИНФО][QUEUES] регистрируем очередь planner
[ИНФО][MISSION PLANNER] создана система планирования заданий
[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][QUEUES] регистрируем очередь communication
[ИНФО][QUEUES] регистрируем очередь control
[ИНФО][CONTROL] создана система управления
[ИНФО][QUEUES] регистрируем очередь navigation
[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][QUEUES] регистрируем очередь cargo
[ИНФО][CARGO] создан компонент грузового отсека, отсек заблокирован
[ИНФО][MISSION PLANNER] старт системы планирования заданий
[ИНФО][CARGO] старт блока грузового отсека
[ИНФО][CONTROL] старт системы управления
[ИНФО][MISSION PLANNER] запрошена новая задача, отправляем получателям
[ИНФО][MISSION PLANNER] новая задача отправлена в коммуникационный шлюз
[ИНФО][CONTROL] установлена новая задача, начинаем следовать по маршруту, текущее время 18:16:27.

На этом модуль 2 завершён, надеемся, у вас всё получилось! 
На что можно ещё обратить вниманием - можно ли как-то улучшить маршрут для ускорения доставки? (без нарушения ПДД) - например, использовать более скоростные трассы или срезать какие-то участки.

Если идей быстрых улучшений нет, то пришло время перейти к самому интересному модулю и познакомиться с киберпрепятствиями!

Для работы над модулем 3 задания откройте блокнот cyberimmunity--autonomous-car-m3.